# IÐN303G Tæknileg kerfi - Sýnidæmi - Stýringar - PID

## Inngangur
Markmiðið með þessu sýnidæmi er að kynnast hönnun PID stýringa.

Gefið er sjálfvirkt stýrikerfi sem má lýsa með eftirfarandi blokkriti.

<div>
<img src="https://github.com/rsaemundsson/idn_303_TS_25/blob/main/automatic_control_system_pid.png?raw=true" alt="Mynd vantar" width=600>

Til stendur að velja stýringu þannig að fasaöryggið sé $\frac{\pi}{3}$.

1. Veljið gildi á $K$ og $T_D$ fyrir PD stýringu sem uppfyllir kröfur um fasaöryggi.  
2. Veljið gildi á $K$, $T_D$, $T_I$ og $T_F$ fyrir PID stýringu sem uppfyllir kröfur um fasaöryggi.
3. Berið saman stýringarnar með því að bera saman tíðnisvörun lykkjunnar, tímasvörun heildarkerfisins og skekkju í jafnvægi eftir breytingar í óskgildi og þrepatruflanir í kerfinu sem á að stýra ($G_p$). Hvað myndi réttlæta að velja PID stýringuna fram yfir PD stýringuna? Væri ástæða til að velja PI stýringu í stað PID stýringarinnar?

In [ ]:
!pip install control

In [ ]:
# Hlaða inn þeim forritasöfnum sem þarf að hlaða inn
import numpy as np
import control as ct
import matplotlib.pyplot as plt

# Fasaöryggi í gráðum

phase_margin_criteria = 60
phase_margin_PI = 60 + 0.06 * 180

# Yfirfærsluföll

G_p=ct.tf(2,[5, 1, 0])
G_a=ct.tf(1,[1,2])

G_K=ct.series(G_a,G_p)

# Stöðugleikagreining

ct.bode(G_K, dB=True)

gm, pm, wcg_init, wcp = ct.margin(G_K)
print("Mögnunaröryggi: ",gm)
print("Fasaöryggi: ",pm)
print("w_pi: ",wcg_init)
print("w_c: ",wcp)

In [ ]:
#PD stýring
T_D = 1.0 / wcg_init
K = 0.2
G_PD = K * ct.tf([T_D, 1],[1])
#print(G_PD)

G_K_PD=ct.series(G_PD, G_a, G_p)
#print(G_K_PD)

# Stöðugleikagreining - Breyti K þar til að kröfur um fasaöryggi séu uppfylltar

ct.bode(G_K_PD, dB=True)

gm, pm, wcg, wcp_PD = ct.margin(G_K_PD)
print("Mögnunaröryggi: ", gm)
print("Fasaöryggi: ",pm)
print("w_pi: ",wcg)
print("w_c: ",wcp_PD)

In [ ]:
#PID stýring
T_D = 1.0 / wcg_init
T_F = 0.3 * T_D
K = 0.083

G_PID_PD = ct.tf([T_D, 1],[T_F, 1])
#print(G_PID_PD)

G_K_1 = K * ct.series(G_PID_PD, G_a, G_p)
#print(G_K_1)

# Stöðugleikagreining - Breyti K þar til fasaöryggi (phase_margin_PI) passar
plt.figure()
ct.bode(G_K_1, dB=True)

gm, pm, wcg, wcp_PI = ct.margin(G_K_1)
print("Markmið um fasaöryggi PI hlutans: ", phase_margin_PI)
print("Mögnunaröryggi: ", gm)
print("Fasaöryggi (PI): ",pm)
print("w_pi: ",wcg)
print("w_c: ",wcp_PI)

T_I = 1.0 / (0.2 * wcp_PI)
G_PID_PI = K * ct.tf([T_I, 1],[T_I, 0])
#print(G_PID_PI)
G_K_PID = ct.series(G_PID_PI, G_PID_PD, G_a, G_p)
#print(G_K_PID)

# Stöðugleikagreining - Geng úr skugga um að fasaöryggi sé rétt
plt.figure()
ct.bode(G_K_PID, dB=True)

gm, pm, wcg, wcp_PID = ct.margin(G_K_PID)
print("Heildin")
print("Mögnunaröryggi: ",gm)
print("Fasaöryggi: ",pm)
print("w_pi: ",wcg)
print("w_c: ",wcp_PID)

In [ ]:
#Samanburður

# Hraði
G_T_PD = ct.feedback(G_K_PD,1)
G_T_PID = ct.feedback(G_K_PID,1)

response_time = 200

[t_PD, y_PD] = ct.step_response(G_T_PD, T=response_time)
plt.plot(t_PD, y_PD, label="PD stýring")

[t_PID, y_PID] = ct.step_response(G_T_PID, T=response_time)
plt.plot(t_PID, y_PID, label="PID stýring")
plt.ylabel("y(t)")
plt.xlabel("t (sek)")
plt.legend()
plt.grid()
plt.title("Svörun y(t) við einingaþrepabreytingu í r(t)")

t_r_PD = 1.4 / wcp_PD
t_r_PID = 1.4 / wcp_PID
print("Ristími PD: %0.2f s" % (t_r_PD))
print("Ristími PID: %0.2f s" % (t_r_PID))

# Skekkja í jafnvægi

# Breyting í óskgildi

G_RE_PD = ct.feedback(1,G_K_PD)
G_RE_PID = ct.feedback(1,G_K_PID)

plt.figure()
[t_PD, y_PD] = ct.step_response(G_RE_PD, T=response_time)
plt.plot(t_PD, y_PD, label="PD stýring")

[t_PID, y_PID] = ct.step_response(G_RE_PID, T=response_time)
plt.plot(t_PID, y_PID, label="PID stýring")
plt.ylabel("e(t)")
plt.xlabel("t (sek)")
plt.legend()
plt.grid()
plt.title("Svörun e(t) við einingaþrepabreytingu í r(t)")

#Þrepatruflun í V

G_VE_PD = ct.feedback(-G_p,-ct.series(G_PD,G_a))
G_VE_PID = ct.feedback(-G_p,-ct.series(G_PID_PI, G_PID_PD, G_a))

#print(G_VE_PID)

plt.figure()
[t_PD, y_PD] = ct.step_response(G_VE_PD, T=response_time)
plt.plot(t_PD, y_PD, label="PD stýring")

[t_PID, y_PID] = ct.step_response(G_VE_PID, T=response_time)
plt.plot(t_PID, y_PID, label="PID stýring")
plt.ylabel("e(t)")
plt.xlabel("t (sek)")
plt.legend()
plt.grid()
plt.title("Svörun e(t) við einingaþrepabreytingu í v(t) (truflun)")

#Tíðnisvörun lykkjunar
plt.figure()
ct.bode(G_K_PD, dB=True, label="PI stýring")
ct.bode(G_K_PID, dB=True, label="PID stýring")